In [6]:
# create a pipeline from the etl_pipeline package
from pipeline_executor import PipelineExecutor

topic = 'quantum'

# create a pipeline executor
pipeline_executor = PipelineExecutor()
quantum_df = pipeline_executor.execute(query=topic, max_articles=2)

# print the first 5 rows of the dataframe
quantum_df.head()

,engine,link,se_description,se_source,n3k_author,n3k_published,title,body
0,Bing,https://phys.org/news/2023-05-google-quantum-a...,Our intuition tells us that it should be impos...,Phys.org,['Google Quantum Ai'],NaN,Google Quantum AI braids non-Abelian anyons fo...,This article has been reviewed according to Sc...
1,Bing,https://www.newscientist.com/article/mg2583438...,In the search for a theory that unites general...,New Scientist,['Author.Fullname'],NaN,How we could discover quantum gravity without ...,In the search for a theory that unites general...
2,Google,https://physics.aps.org/articles/v16/s71,1 day ago,PHYSICS - APS.org,['Marric Stephens'],2023-05-11 00:00:00,Proof That a Complex Quantum Network Is Truly ...,Synopsis\n\nProof That a Complex Quantum Netwo...
3,Google,https://physicsworld.com/a/germany-reveals-e3b...,2 days ago,Physics World,[],2023-05-10 13:31:19+00:00,Germany reveals €3bn plan to build a quantum c...,"The German quantum computer, to be built by 20..."


In [7]:
# TODO extract entities from the dataframe descritpion, title, and body columns
# entities = quantum_df['description'].apply(lambda x: extract_entities(x))

entities = ['Microsoft', 'Apple']

In [11]:
# for each entity run the pipeline (adding "quantum" to the query) and store the results in a dictionary
entity_results = {}
for entity in entities:
    entity_results[entity] = pipeline_executor.execute(query=f'{entity} AND {topic}', max_articles=2)


In [14]:
# check the results for the first entity
entity_results['Microsoft'].head()

,engine,link,se_description,se_source,n3k_author,n3k_published,title,body
0,Bing,https://www.scientificamerican.com/article/phy...,Exotic particles called nonabelions could fix ...,Scientific American,"['Davide Castelvecchi Nature Magazine', 'About...",NaN,Physicists Create Long-Sought Topological Quan...,The coat of arms of Italy’s aristocratic House...
1,Yahoo,https://seekingalpha.com/article/4603046-micro...,Quantum computing is a disruptive technology t...,Seeking Alpha,['Aseity Research'],2023-05-11 11:03:43-04:00,Microsoft Stock: Leading The Quantum Computing...,Bartlomiej Wroblewski\n\nQuantum computing is ...
2,Yahoo,https://www.morningstar.com/news/pr-newswire/2...,Quantum Computing Inc. initiates commercializa...,Morningstar,[],2023-05-11 15:01:00-05:00,Quantum Computing Inc Announces First Quarter ...,Quantum Computing Inc. initiates commercializa...
3,Google,https://cloudblogs.microsoft.com/quantum/2023/...,"At Microsoft, we believe that it will take the...",Microsoft Cloud Blogs,['Matthias Troyer'],2023-05-01 00:00:00,Quantum Advantage: Hope and Hype - Microsoft A...,Efforts to research and develop quantum comput...
4,Google,https://seekingalpha.com/article/4603046-micro...,Microsoft's unique approach with topological q...,Seeking Alpha,['Aseity Research'],2023-05-11 11:03:43-04:00,Microsoft Stock: Leading The Quantum Computing...,Bartlomiej Wroblewski\n\nQuantum computing is ...


In [15]:
# TODO summarize the results